In [12]:
import scipy.io.wavfile as wavio
import numpy as np
import scipy.signal as sig
import IPython


sample_rate = 48000
duration = 2
# C4 @ 260Hz
c4 = np.linspace(0, 260 * duration, sample_rate * duration)
c4 = np.sin(2 * np.pi * c4)
# E4 @ 330Hz
e4 = np.linspace(0, 330 * duration, sample_rate * duration)
e4 = np.sin(2 * np.pi * e4)
# G4 @ 392Hz
g4 = np.linspace(0, 392 * duration, sample_rate * duration)
g4 = np.sin(2 * np.pi * g4)
# Triad
harmo_i = c4 + e4 + g4
IPython.display.display(IPython.display.Audio(harmo_i, rate=sample_rate))

In [19]:
# DTMF
duration = 1/4
# Columns
hz_1209 = np.linspace(0, int(1209 * duration), int(sample_rate * duration))
hz_1209 = np.sin(2 * np.pi * hz_1209)
hz_1336 = np.linspace(0, int(1336 * duration), int(sample_rate * duration))
hz_1336 = np.sin(2 * np.pi * hz_1336)
hz_1477 = np.linspace(0, int(1477 * duration), int(sample_rate * duration))
hz_1477 = np.sin(2 * np.pi * hz_1477)
# Rows
hz_697 = np.linspace(0, int(697 * duration), int(sample_rate * duration))
hz_697 = np.sin(2 * np.pi * hz_697)
hz_770 = np.linspace(0, int(770 * duration), int(sample_rate * duration))
hz_770 = np.sin(2 * np.pi * hz_770)
hz_852 = np.linspace(0, int(852 * duration), int(sample_rate * duration))
hz_852 = np.sin(2 * np.pi * hz_852)
hz_941 = np.linspace(0, int(941 * duration), int(sample_rate * duration))
hz_941 = np.sin(2 * np.pi * hz_941)
# Numbers
one = hz_1209 + hz_697
two = hz_1336 + hz_697
three = hz_1477 + hz_697
four = hz_1209 + hz_770
five = hz_1336 + hz_770
six = hz_1477 + hz_770
seven = hz_1209 + hz_852
eight = hz_1336 + hz_852
nine = hz_1477 + hz_852
star = hz_1209 + hz_941
zero = hz_1336 + hz_941
sharp = hz_1477 + hz_941
number = np.hstack((one, two, three, four, five, six, seven, eight, nine, star, zero, sharp))
IPython.display.display(IPython.display.Audio(number, rate=sample_rate))
number = np.hstack((two, one, seven, four, five, three, nine, one, zero, seven))
IPython.display.display(IPython.display.Audio(number, rate=sample_rate))